In [1]:
from gurobipy import *
import numpy as np
import pandas as pd

data = pd.read_excel (r'C:\Users\karbassi\Python\Test_1.xlsx', sheet_name=0)

m = Model()

# data sets

supplycountry_names = data['SupplyCountry'].tolist() # names of energy supply countries: ["Angola","Algeria", ..]
supplycountry_indices = range(len(supplycountry_names)) # number of supply countries: [0,1,..]
demandcountry_names = data['DemandCountry'].tolist() # names of energy demand countries: ["Angola","Algeria", ..]
demandcountry_indices = range(len(demandcountry_names)) # number of demand countries: [0,1,..]
technology_names = data['Technologies'].tolist() # names of Technologies: ["Solar","Wind"]
technology_indices = range(len(technology_names)) # number of Technologies: ["Solar","Wind"]

# define adjacency matrix for available transport paths between countries

adjacency = np.array([[1,1,1],
                      [1,1,1],
                      [1,1,1]])

#adjacency = np.repeat(adjacency[:, :, np.newaxis], 3, axis=2) #assuming if transport is possible, it is possible for all technologies

# parameters:

supplysheet = pd.read_excel(r'C:\Users\karbassi\Python\Test_1.xlsx', sheet_name=1)
gencap = [] # generation capacity of supply country i for technology t
for i in supplycountry_names:
    gencap.append(supplysheet[(supplysheet['SupplyCountry'] == i)]['Supply'].tolist()) 
    
demandsheet = pd.read_excel(r'C:\Users\karbassi\Python\Test_1.xlsx', sheet_name=2)
d = demandsheet['Demand'].tolist()  # demand of demand country j (MW)

gencostsheet = pd.read_excel(r'C:\Users\karbassi\Python\Test_1.xlsx', sheet_name=3) # generation cost in supply country i using technology t
gencost = [] 
for i in supplycountry_names:
    gencost.append(gencostsheet[(gencostsheet['SupplyCountry'] == i)]['genCost'].tolist())
                   
# generation cost of each unit (e.g. MW) energy carrier in supply country i (million $/MW)

tcsheet = pd.read_excel (r'C:\Users\karbassi\Python\Test_1.xlsx', sheet_name=4)  # transport cost from supply country i to demand country j (million $)
tc = []
for i in supplycountry_names:
    tc.append(tcsheet[(tcsheet['SupplyCountry'] == i)]['transCost'].tolist())

# decision variables

x_elec = m.addVars(supplycountry_indices, demandcountry_indices, technology_indices, name="transport")   # xijt = (quantity) amount of electricty transported from supply country i to demand country j with technology t (MW)

gen_elec=m.addVars(supplycountry_indices, technology_indices, name='generation')

cons_elec=m.addVars(supplycountry_indices, technology_indices, name='consumption')

m.update()

# objective function
m.setObjective(sum(gencost[i][t]*gen_elec[i,t] for i in supplycountry_indices for t in technology_indices) + sum(tc[i][j]*x_elec[i,j,t] for i in supplycountry_indices for j in demandcountry_indices for t in technology_indices), GRB.MINIMIZE) # economic objective


# constraints

# 1.amount of electricity transferred to each demand country should be smaller or equal to the generation capacity of the supply country
m.addConstrs(
    gen_elec[i,t] <= gencap[i][t] 
    for i in supplycountry_indices for t in technology_indices
)

# 2.demand should be fulfilled by the electricity transferred from each supply country
m.addConstrs(
    sum(cons_elec[j,t] for t in technology_indices) == d[j]
    for j in demandcountry_indices
)

#3.each country should be able to satisfy 50% of its own demand

for i in supplycountry_indices:
    m.addConstr(sum(gen_elec[i,t] for t in technology_indices) >= 0.5*d[i])

#4. each country can export energy to neighboring countries 

m.addConstrs(x_elec[i, j, t] == 0 for i in supplycountry_indices for j in demandcountry_indices if adjacency[i, j]==0)

# for i in supplycountr:
#     for j .. :
#         if :
#             m.addConstr


# Material Balance Constraint:

m.addConstrs(gen_elec[i,t]-cons_elec[i,t]+ sum(x_elec[j,i,t]-x_elec[i,j,t] for j in demandcountry_indices if i!=j)==0
            for i in supplycountry_indices for t in technology_indices)


# m.addConstr(x_elec[0,1,2] == 20)

# optimization of model
m.optimize()

if m.status == GRB.OPTIMAL:
    print(f"Optimal solutions of transport cost: {m.ObjVal} ")
    
    m.getVars()

    for i in supplycountry_indices:
        summation = 0
        for j in demandcountry_indices:
            for t in technology_indices:
                summation += x_elec[i,j,t].x 
        if (summation> 0):
            print(f"Supply Country {i} provides electricity")

    print("Transport amount from supply country i to demand country j: ")
    for i in supplycountry_indices:
        for j in demandcountry_indices:
            for t in technology_indices:
                if (x_elec[i,j,t].x > 0):
                    print(f"from supply country {i} to demandcountry {j} with technology {t}: {x_elec[i,j,t].x}")
    for i in supplycountry_indices:
        total = 0
        for t in technology_indices:
            print (f"generation capacity of country {i} for technology {t} is: {gencap[i][t]}")
            total += gencap[i][t]
        print (f"total generation capacity of country {i} is: {total}")
else:
    print(m.status)

m.printAttr("X")
                    
m.write("simplemodel.lp")




Academic license - for non-commercial use only - expires 2021-04-02
Using license file C:\Users\karbassi\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 24 rows, 45 columns and 81 nonzeros
Model fingerprint: 0x999b5b28
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 6e+02]
Presolve removed 9 rows and 9 columns
Presolve time: 0.00s
Presolved: 15 rows, 36 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.062500e+01   0.000000e+00      0s
       8    2.1500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds
Optimal objective  2.150000000e+03
Optimal solutions of transport cost: 2150.0 
Transport amount from supply country i to demand country j: 
generation capacity of coun

In [ ]:
tcsheet[(tcsheet['SupplyCountry'] == 'Angola')]

In [ ]:
print (tcsheet)

In [6]:
for i in supplycountry_indices:
    for j in demandcountry_indices:
        for t in technology_indices:
            print(x_elec[i,j,t].x)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
